## Loading the required modules

In [3]:
%pylab inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import os 
import pysumma as ps
import pysumma.plotting as psp
import matplotlib.pyplot as plt
from pathlib import Path

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<br>

## Instantiating a simulation object


In [4]:
# Define location of .exe and file manager
summa_exe = '/Users/dcasson/GitHub/summa/bin/summa.exe'
reynolds_data_path = '/Users/dcasson/GitHub/pysumma/tutorial/data/reynolds/'
confluence_station_data_path = '/Users/dcasson/Data/snow_lse/domain_aiken_lake_pillow/'
reynolds_file_manager = Path(reynolds_data_path, 'file_manager.txt')
confluence_file_manager = Path(confluence_station_data_path, 'fileManager.txt')

## Create simulation objects

Note the following updates are needed:
- Update the file manager to point to the correct directory
- Update the output control to be compatiable with pysumma

In [25]:
# Create a model instance
reynolds_sim = ps.Simulation(summa_exe, reynolds_file_manager)
confluence_sim = ps.Simulation(summa_exe, confluence_file_manager)

### File manager
Let's take a look at what the file manager text file actually contains:

In [27]:
print(reynolds_sim.manager)
print(confluence_sim.manager)

controlVersion                       'SUMMA_FILE_MANAGER_V3.0.0'
simStartTime                         '1999-10-01 01:00'
simEndTime                           '2002-09-30 23:00'
tmZoneInfo                           'localTime'
settingsPath                         '/Users/dcasson/GitHub/pysumma/tutorial/data/reynolds/settings/'
forcingPath                          '/Users/dcasson/GitHub/pysumma/tutorial/data/reynolds/forcing/'
outputPath                           '/Users/dcasson/GitHub/pysumma/tutorial/data/reynolds/output/'
decisionsFile                        'snow_zDecisions.txt'
outputControlFile                    'snow_zOutputControl.txt'
globalHruParamFile                   'snow_zLocalParamInfo.txt'
globalGruParamFile                   'snow_zBasinParamInfo.txt'
attributeFile                        'snow_zLocalAttributes.nc'
trialParamFile                       'snow_zParamTrial.nc'
forcingListFile                      'forcing_file_list.txt'
initConditionFile                    

In [56]:
def compare_simulation_options(sim1, sim2, attr_name, sim1_name="sim1", sim2_name="sim2"):
    """
    Compare the specified attribute (e.g., decisions or global_hru_params) between two simulations.
    
    Parameters:
    -----------
    sim1 : Simulation object
        First simulation to compare
    sim2 : Simulation object
        Second simulation to compare
    attr_name : str
        Name of the attribute to compare (e.g., 'decisions' or 'global_hru_params')
    sim1_name : str, optional
        Name of the first simulation for display purposes
    sim2_name : str, optional
        Name of the second simulation for display purposes
    """
    print(f"Comparing '{attr_name}' between {sim1_name} and {sim2_name}:")
    print("-" * 80)

    # Get the attribute (assumed to be a list-like object with 'options')
    attr1 = getattr(sim1, attr_name)
    attr2 = getattr(sim2, attr_name)

    # Get all unique option names
    keys1 = {opt.name for opt in attr1.options}
    keys2 = {opt.name for opt in attr2.options}
    all_keys = sorted(keys1.union(keys2))

    differences_found = False

    for key in all_keys:
        opt1 = next((opt for opt in attr1.options if opt.name == key), None)
        opt2 = next((opt for opt in attr2.options if opt.name == key), None)

        if opt1 is None:
            print(f"{key} only in {sim2_name}: {opt2.value}")
            differences_found = True
        elif opt2 is None:
            print(f"{key} only in {sim1_name}: {opt1.value}")
            differences_found = True
        elif str(opt1.value) != str(opt2.value):
            print(f"{key}:")
            print(f"  {sim1_name}: {opt1.value}")
            print(f"  {sim2_name}: {opt2.value}")
            differences_found = True

    if not differences_found:
        print(f"No differences found in '{attr_name}' between the two simulations.")

# Compare decisions
compare_simulation_options(reynolds_sim, confluence_sim, sim1_name="reynolds_sim", sim2_name="confluence_sim", attr_name="decisions")

# Compare global HRU parameters
compare_simulation_options(reynolds_sim, confluence_sim, sim1_name="reynolds_sim", sim2_name="confluence_sim", attr_name="global_hru_params")


Comparing 'decisions' between reynolds_sim and confluence_sim:
--------------------------------------------------------------------------------
alb_method:
  reynolds_sim: varDecay
  confluence_sim: conDecay
hc_profile:
  reynolds_sim: pow_prof
  confluence_sim: constant
rootProfil only in reynolds_sim: powerLaw
snowDenNew only in reynolds_sim: constDens
subRouting:
  reynolds_sim: timeDlay
  confluence_sim: qInstant
veg_traits:
  reynolds_sim: CM_QJRMS1988
  confluence_sim: Raupach_BLM1994
vegeParTbl:
  reynolds_sim: USGS
  confluence_sim: MODIFIED_IGBP_MODIS_NOAH
Comparing 'global_hru_params' between reynolds_sim and confluence_sim:
--------------------------------------------------------------------------------
albedoMax:
  reynolds_sim: [0.84, 0.7, 0.95]
  confluence_sim: [0.84, 0.7, 0.9]
albedoMinWinter:
  reynolds_sim: [0.55, 0.6, 1.0]
  confluence_sim: [0.55, 0.55, 1.0]
aquiferBaseflowRate:
  reynolds_sim: [2.0, 1.0, 10.0]
  confluence_sim: [0.1, 0.0, 0.1]
canopyWettingExp:
  re